In [2]:
# 检索包含"我这辈子"的评论及其相似度特征
import pandas as pd
import numpy as np
import os

# 切换到项目根目录
os.chdir(r'D:\010_CodePrograms\L\LLM_su7')

# 加载原始数据
print("加载数据...")
train_df = pd.read_pickle('train.pkl')
val_df = pd.read_pickle('val.pkl')
test_df = pd.read_pickle('test.pkl')

# 加载时间密度特征
train_td = pd.read_pickle('train_time_density.pkl')
val_td = pd.read_pickle('val_time_density.pkl')
test_td = pd.read_pickle('test_time_density.pkl')

# 合并所有数据
train_df['数据集'] = 'train'
val_df['数据集'] = 'val'
test_df['数据集'] = 'test'
all_df = pd.concat([train_df, val_df, test_df], ignore_index=True)

# 合并时间密度特征
all_td = pd.concat([train_td, val_td, test_td], ignore_index=True)
print(f"总数据量: {len(all_df):,} 条")
print(f"时间密度特征: {len(all_td):,} 条")

加载数据...
总数据量: 271,452 条
时间密度特征: 271,452 条


In [3]:
# 将时间密度特征合并到原数据
all_df = all_df.merge(all_td, on='序号', how='left')

# 筛选包含"我这辈子"的评论
keyword = '我这辈子'
mask = all_df['评论文案'].str.contains(keyword, na=False)
filtered_df = all_df[mask].copy()
print(f'包含"{keyword}"的评论数量: {len(filtered_df):,} 条')

# 按时间顺序索引排序
filtered_df = filtered_df.sort_values('时间顺序索引').reset_index(drop=True)

包含"我这辈子"的评论数量: 17 条


In [4]:
# 显示结果
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', None)

# 选择需要显示的列
display_cols = ['时间顺序索引', '序号', '发布时间', '评论文案', '最大相似度', '重复次数', '数据集']
result = filtered_df[display_cols]

print(f'按时间顺序显示包含"{keyword}"的评论:\n')
for idx, row in result.iterrows():
    print(f"[时间索引 {row['时间顺序索引']:>6}] 序号: {row['序号']:>6} | 相似度: {row['最大相似度']:.4f} | 重复次数: {row['重复次数']:>4} | {row['数据集']}")
    print(f"  发布时间: {row['发布时间']}")
    comment = str(row['评论文案'])
    print(f"  评论: {comment[:150]}{'...' if len(comment) > 150 else ''}")
    print()

按时间顺序显示包含"我这辈子"的评论:

[时间索引  32062] 序号: 145210 | 相似度: 0.9020 | 重复次数:    1 | train
  发布时间: 2025-03-31 16:12:14
  评论: 这，可能是我这辈子开过加速最快情绪价值最高的车

[时间索引  58320] 序号:   3072 | 相似度: 0.9110 | 重复次数:    2 | train
  发布时间: 2025-04-01 23:06:29
  评论: 我这辈子第一次听说，出了交通事故在第一时间不找警察，不找保险，而是找汽车商家的

[时间索引  59338] 序号: 256532 | 相似度: 0.9984 | 重复次数:    4 | test
  发布时间: 2025-04-01 23:22:36
  评论: 我这辈子第一次听说，出了交通事故在第一时间不找警察，不找保险公司，而是找汽车商家的

[时间索引  59448] 序号: 118947 | 相似度: 0.9981 | 重复次数:    3 | train
  发布时间: 2025-04-01 23:24:21
  评论: 我这辈子第一次听说，出了交通事故在第一时间不找警察，不找保险，而是找汽车商家的

[时间索引  59736] 序号:  16213 | 相似度: 1.0000 | 重复次数:    7 | train
  发布时间: 2025-04-01 23:29:18
  评论: 我这辈子第一次听说，出了交通事故在第一时间不找警察，不找保险，而是找汽车商家的

[时间索引  61065] 序号: 199354 | 相似度: 0.9662 | 重复次数:    4 | train
  发布时间: 2025-04-01 23:57:39
  评论: 是这样的//@身骑白马:我这辈子第一次听说，出了交通事故在第一时间不找警察，不找保险，而是找汽车商家的

[时间索引  62879] 序号: 262717 | 相似度: 0.8693 | 重复次数:    0 | test
  发布时间: 2025-04-02 00:48:58
  评论: 是的，调查结果还没出来，就让人家车企负责[允悲]我这辈子第一次听说，出了交通事故在第一时间不找警察，不找保险，而是找汽车商家的，目的地太明显了，一堆傻子还在嘿

In [5]:
# 统计信息
print("=" * 60)
print(f"统计信息:")
print(f"  总数量: {len(filtered_df)} 条")
print(f"  最大相似度 - 均值: {filtered_df['最大相似度'].mean():.4f}, 最大: {filtered_df['最大相似度'].max():.4f}, 最小: {filtered_df['最大相似度'].min():.4f}")
print(f"  重复次数 - 均值: {filtered_df['重复次数'].mean():.2f}, 最大: {filtered_df['重复次数'].max()}, 最小: {filtered_df['重复次数'].min()}")
print(f"  时间范围: {filtered_df['发布时间'].min()} ~ {filtered_df['发布时间'].max()}")

统计信息:
  总数量: 17 条
  最大相似度 - 均值: 0.9191, 最大: 1.0000, 最小: 0.7555
  重复次数 - 均值: 3.76, 最大: 12, 最小: 0
  时间范围: 2025-03-31 16:12:14 ~ 2025-04-12 10:01:06


In [6]:
# 加载MinHash特征进行对比
train_td_mh = pd.read_pickle('train_time_density_minhash.pkl')
val_td_mh = pd.read_pickle('val_time_density_minhash.pkl')
test_td_mh = pd.read_pickle('test_time_density_minhash.pkl')
all_td_mh = pd.concat([train_td_mh, val_td_mh, test_td_mh], ignore_index=True)

# 重新加载原始数据
train_df = pd.read_pickle('train.pkl')
val_df = pd.read_pickle('val.pkl') 
test_df = pd.read_pickle('test.pkl')
train_df['数据集'] = 'train'
val_df['数据集'] = 'val'
test_df['数据集'] = 'test'
all_df_new = pd.concat([train_df, val_df, test_df], ignore_index=True)

# 合并BGE和MinHash特征
all_df_new = all_df_new.merge(all_td, on='序号', how='left', suffixes=('', '_bge'))
all_df_new = all_df_new.merge(all_td_mh, on='序号', how='left', suffixes=('_bge', '_minhash'))

print("特征合并完成")
print(f"BGE列: 最大相似度_bge, 重复次数_bge")
print(f"MinHash列: 最大相似度_minhash, 重复次数_minhash")

特征合并完成
BGE列: 最大相似度_bge, 重复次数_bge
MinHash列: 最大相似度_minhash, 重复次数_minhash


In [7]:
# 筛选包含"我这辈子"的评论并对比两种方法
keyword = '我这辈子'
mask = all_df_new['评论文案'].str.contains(keyword, na=False)
filtered_compare = all_df_new[mask].copy()
filtered_compare = filtered_compare.sort_values('时间顺序索引_bge').reset_index(drop=True)

print(f'包含"{keyword}"的评论: {len(filtered_compare)} 条\n')
print("=" * 100)
print(f"{'时间索引':>8} | {'BGE相似度':>10} | {'BGE重复':>8} | {'MH相似度':>10} | {'MH重复':>8} | 评论摘要")
print("=" * 100)

for idx, row in filtered_compare.iterrows():
    time_idx = int(row['时间顺序索引_bge'])
    bge_sim = row['最大相似度_bge']
    bge_cnt = int(row['重复次数_bge'])
    mh_sim = row['最大相似度_minhash']
    mh_cnt = int(row['重复次数_minhash'])
    comment = str(row['评论文案'])[:]
    
    print(f"{time_idx:>8} | {bge_sim:>10.4f} | {bge_cnt:>8} | {mh_sim:>10.4f} | {mh_cnt:>8} | {comment}...")

包含"我这辈子"的评论: 17 条

    时间索引 |     BGE相似度 |    BGE重复 |      MH相似度 |     MH重复 | 评论摘要
   32062 |     0.9020 |        1 |     0.0703 |        0 | 这，可能是我这辈子开过加速最快情绪价值最高的车...
   58320 |     0.9110 |        2 |     0.0938 |        0 | 我这辈子第一次听说，出了交通事故在第一时间不找警察，不找保险，而是找汽车商家的...
   59338 |     0.9984 |        4 |     0.8828 |        1 | 我这辈子第一次听说，出了交通事故在第一时间不找警察，不找保险公司，而是找汽车商家的...
   59448 |     0.9981 |        3 |     1.0000 |        2 | 我这辈子第一次听说，出了交通事故在第一时间不找警察，不找保险，而是找汽车商家的...
   59736 |     1.0000 |        7 |     1.0000 |        3 | 我这辈子第一次听说，出了交通事故在第一时间不找警察，不找保险，而是找汽车商家的...
   61065 |     0.9662 |        4 |     0.6797 |        4 | 是这样的//@身骑白马:我这辈子第一次听说，出了交通事故在第一时间不找警察，不找保险，而是找汽车商家的...
   62879 |     0.8693 |        0 |     0.4453 |        0 | 是的，调查结果还没出来，就让人家车企负责[允悲]我这辈子第一次听说，出了交通事故在第一时间不找警察，不找保险，而是找汽车商家的，目的地太明显了，一堆傻子还在嘿嘿嘿...
   66067 |     0.9189 |        5 |     0.7266 |        5 | @雷军 我这辈子第一次听说，出了交通事故在第一时间不找警察，不找保险，而是找汽车商家的………人不能太老实...
   66115 |     0.9820 |        8 |     0.9297 | 

In [8]:
# 统计对比
print("\n" + "=" * 60)
print("两种方法对比统计:")
print("=" * 60)
print(f"\n【BGE语义相似度】(阈值=0.9)")
print(f"  平均最大相似度: {filtered_compare['最大相似度_bge'].mean():.4f}")
print(f"  平均重复次数: {filtered_compare['重复次数_bge'].mean():.2f}")
print(f"  检测到重复的评论数: {(filtered_compare['重复次数_bge'] > 0).sum()}")

print(f"\n【MinHash Jaccard相似度】(阈值=0.5, N-gram=3)")
print(f"  平均最大相似度: {filtered_compare['最大相似度_minhash'].mean():.4f}")
print(f"  平均重复次数: {filtered_compare['重复次数_minhash'].mean():.2f}")
print(f"  检测到重复的评论数: {(filtered_compare['重复次数_minhash'] > 0).sum()}")

print("\n" + "=" * 60)
print("分析结论:")
print("=" * 60)
print("""
1. BGE语义相似度能够捕捉语义相近但文字不完全相同的评论
   - 对于完全相同的评论，相似度=1.0
   - 对于稍有改动的评论（如添加表情、引用），相似度也很高(>0.9)
   
2. MinHash Jaccard相似度基于字符N-gram匹配
   - 只有文字高度重合时才会有高相似度
   - 对于添加了前缀/后缀的评论，相似度会降低
   - 计算速度更快，适合大规模去重任务
   
3. 对于"我这辈子第一次听说..."这类复制粘贴的评论：
   - BGE能识别更多变体（如添加@雷军、表情、引用等）
   - MinHash只能识别几乎完全相同的文本
""")


两种方法对比统计:

【BGE语义相似度】(阈值=0.9)
  平均最大相似度: 0.9191
  平均重复次数: 3.76
  检测到重复的评论数: 13

【MinHash Jaccard相似度】(阈值=0.5, N-gram=3)
  平均最大相似度: 0.5175
  平均重复次数: 1.76
  检测到重复的评论数: 8

分析结论:

1. BGE语义相似度能够捕捉语义相近但文字不完全相同的评论
   - 对于完全相同的评论，相似度=1.0
   - 对于稍有改动的评论（如添加表情、引用），相似度也很高(>0.9)

2. MinHash Jaccard相似度基于字符N-gram匹配
   - 只有文字高度重合时才会有高相似度
   - 对于添加了前缀/后缀的评论，相似度会降低
   - 计算速度更快，适合大规模去重任务

3. 对于"我这辈子第一次听说..."这类复制粘贴的评论：
   - BGE能识别更多变体（如添加@雷军、表情、引用等）
   - MinHash只能识别几乎完全相同的文本

